In [19]:
import os
import sys
import torch
import zipfile
from google.colab import drive
import matplotlib.pyplot as plt

batch_size = 5
max_epochs = 400
learning_rate = 0.01
outp_size = [224,224]
load_prev_weights = True
cuda = torch.cuda.is_available()

# Paths
drive.mount('/content/drive/')

Train_ann_path = '/home/Coin_Dataset/Annotations/training.csv'
Train_img_path = '/home/Coin_Dataset/Images/training/'

Valid_ann_path = '/home/Coin_Dataset/Annotations/validation.csv'
Valid_img_path = '/home/Coin_Dataset/Images/validation/'

model_save_location = '/content/drive/My Drive/Colab Notebooks/Coin_CNN/my_model.pth'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [20]:
try:
  os.listdir('/home/Coin_Dataset/Images/training')
except:
  zip_ref = zipfile.ZipFile('/content/drive/My Drive/Colab Notebooks/Datasets/Coin_Dataset.zip')
  zip_ref.extractall('/home')
  zip_ref.close()

sys.path.append('/content/drive/My Drive/Colab Notebooks/Coin_CNN/')

import MobileNetV2
import my_loss
import Dataset

In [ ]:
if __name__ == '__main__':
    train_dataset = Dataset.Make(Train_img_path, Train_ann_path, output_size=outp_size)
    valid_dataset = Dataset.Make(Valid_img_path, Valid_ann_path, output_size=outp_size)
    train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True, num_workers=3)
    valid_dataloader = torch.utils.data.DataLoader(valid_dataset,batch_size=batch_size,shuffle=False,num_workers=1)

    #model = MobileNet.Make()
    model = MobileNetV2.Make(N_classes=5,pretrained=True)
    if load_prev_weights:
        try:
            model.load_state_dict(torch.load(model_save_location))
            print('Weights loaded successfuly!')
        except:
            print('Weights not loaded!')
            raise AssertionError
    if cuda: model = model.cuda()
    My_loss = my_loss.Make(Type='MSE',cuda=cuda)

    optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

    for epoch in range(max_epochs):
        epoch_train_loss = 0
        epoch_valid_loss = 0

        model.train()
        for data in train_dataloader:
            optimizer.zero_grad()
            
            images = data['image']
            targets = data['value'].long()
            if cuda:
                images = images.cuda()
                targets= targets.cuda()

            outputs= model(images)

            loss = My_loss.compute(targets,outputs)
            
            loss.backward()
            optimizer.step()
            epoch_train_loss += loss

        model.eval()
        with torch.no_grad():
            for data in valid_dataloader:
                images = data['image']
                targets = data['value'].long()
                if cuda:
                    images = images.cuda()
                    targets= targets.cuda()

                outputs= model(images)
                epoch_valid_loss += My_loss.compute(targets,outputs)

        epoch_train_loss /= len(train_dataloader)
        epoch_valid_loss /= len(valid_dataloader)

        torch.save(model.state_dict(), model_save_location)
        print(f'Epoch: {epoch+1}/{max_epochs} | Training Loss: {epoch_train_loss:.4f} | Validation Loss: {epoch_valid_loss:.4f}')

Weights loaded successfuly!
Epoch: 1/400 | Training Loss: 0.0255 | Validation Loss: 0.0067
Epoch: 2/400 | Training Loss: 0.0212 | Validation Loss: 0.0069


In [8]:
import os
print(os.listdir('/home'))

['coins_data2']
